### Setup

#### Load "dotenv" extension to use enviroment variables in the Notebook

In [1]:
%load_ext dotenv
%dotenv

#### Load required libraries

In [5]:
from sqlalchemy import create_engine, text
import psycopg2 as ps
import pandas as pd
import os

#### Setup database connection

Make sure you create a `.env` file in the same directory as this notebook.

Your `.env` file should contain the definition of the following variables:
- DB_HOSTNAME
- DB_PORT
- DB_NAME
- DB_USER
- DB_PASS


Example:
```
DB_HOSTNAME = "localhost"
DB_PORT = 5432
DB_NAME = "retail_db"
DB_USER = "retail_user"
DB_PASS = "retailpassword"
```

In [6]:
DB_HOST = os.getenv("DB_HOSTNAME")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
    
DB_CONNECTION_URI = "postgresql://%s:%s@%s:%s/%s" % (DB_USER, DB_PASS, DB_HOST, DB_PORT, DB_NAME)

# Create SQL engine with SQLAlchemy
sql_engine = create_engine(DB_CONNECTION_URI)

> You can now execute queries to the database with pandas!

# Exercises - Filtering and Aggregation

Let us take care of exercises related to filtering and aggregations using SQL:

- Get all the courses which are in `inactive` or `draft` status.
- Get all the courses which are related to `Python` or `Scala`
- Get count of courses by `course_status`
- Get count of published courses by `course_author`
- Get all the `Python`or `Scala` related courses in `draft` status.
- Get the author and count where the author have more than **one published** course.

## Create and populate the table we are working with.

In [7]:
sql_engine.execute('DROP TABLE IF EXISTS courses')

sql_engine.execute('''
    CREATE TABLE courses(
        course_id SERIAL PRIMARY KEY,
        course_name VARCHAR(60),
        course_author VARCHAR(40),
        course_status VARCHAR(9),
        course_published_dt DATE
    )
''')

sql_engine.execute('''
    INSERT INTO courses
        (course_name, course_author, course_status, course_published_dt)
    VALUES
        ('Programming using Python', 'Bob Dillon', 'published', '2020-09-30'),
        ('Data Engineering using Python', 'Bob Dillon', 'published', '2020-07-15'),
        ('Data Engineering using Scala', 'Elvis Presley', 'draft', null),
        ('Programming using Scala', 'Elvis Presley', 'published', '2020-05-12'),
        ('Programming using Java', 'Mike Jack', 'inactive', '2020-08-10'),
        ('Web Applications - Python Flask', 'Bob Dillon', 'inactive', '2020-07-20'),
        ('Web Applications - Java Spring', 'Bob Dillon', 'draft', null),
        ('Pipeline Orchestration - Python', 'Bob Dillon', 'draft', null),
        ('Streaming Pipelines - Python', 'Bob Dillon', 'published', '2020-10-05'),
        ('Web Applications - Scala Play', 'Elvis Presley', 'inactive', '2020-09-30'),
        ('Web Applications - Python Django', 'Bob Dillon', 'published', '2020-06-23'),
        ('Server Automation - Ansible', 'Uncle Sam', 'published', '2020-07-05')
''')

/var/folders/xn/qgw2zr6n09nfvhn94_y0cwy00000gn/T/ipykernel_71806/3835328181.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  sql_engine.execute('DROP TABLE IF EXISTS courses')


## Solutions

**NOTE:** You can run queries using pandas and SQLAlchemy as it follows:

```python
sql_df = pd.read_sql_query(
    '''
        SELECT *
        FROM
            courses
        ORDER BY
            course_id;
    ''',
    con = sql_engine
)
 
# Show query results
sql_df
```

### Exercise 1

- Get all the courses which are in inactive or draft status.

In [17]:
sql_df = pd.read_sql_query(
    '''
        SELECT *
        FROM
            courses
        WHERE
            course_status IN ('inactive', 'draft')
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,course_id,course_name,course_author,course_status,course_published_dt
0,3,Data Engineering using Scala,Elvis Presley,draft,None
1,5,Programming using Java,Mike Jack,inactive,2020-08-10
2,6,Web Applications - Python Flask,Bob Dillon,inactive,2020-07-20
3,7,Web Applications - Java Spring,Bob Dillon,draft,None
4,8,Pipeline Orchestration - Python,Bob Dillon,draft,None
5,10,Web Applications - Scala Play,Elvis Presley,inactive,2020-09-30


### Exercise 2

- Get all the courses which are related to `Python` or `Scala`

In [20]:
sql_df = pd.read_sql_query(
    '''
        SELECT *
        FROM
            courses
        WHERE
            course_name ILIKE '%%Python%%' OR course_name ILIKE '%%Scala%%';
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,course_id,course_name,course_author,course_status,course_published_dt
0,1,Programming using Python,Bob Dillon,published,2020-09-30
1,2,Data Engineering using Python,Bob Dillon,published,2020-07-15
2,3,Data Engineering using Scala,Elvis Presley,draft,None
3,4,Programming using Scala,Elvis Presley,published,2020-05-12
4,6,Web Applications - Python Flask,Bob Dillon,inactive,2020-07-20
5,8,Pipeline Orchestration - Python,Bob Dillon,draft,None
6,9,Streaming Pipelines - Python,Bob Dillon,published,2020-10-05
7,10,Web Applications - Scala Play,Elvis Presley,inactive,2020-09-30
8,11,Web Applications - Python Django,Bob Dillon,published,2020-06-23


### Exercise 3

- Get count of courses by `course_status`

In [24]:
sql_df = pd.read_sql_query(
    '''
        SELECT
            course_status,
            COUNT(*) AS count
        FROM
            courses
        GROUP BY
            course_status;
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,course_status,count
0,published,6
1,inactive,3
2,draft,3


### Exercise 4

- Get count of published courses by `course_author`

In [42]:
sql_df = pd.read_sql_query(
    '''
        SELECT
            course_author,
            COUNT(course_status) AS published_courses
        FROM
            courses
        WHERE
            course_status = 'published'
        GROUP BY
            course_author,
            course_status;
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,course_author,published_courses
0,Bob Dillon,4
1,Elvis Presley,1
2,Uncle Sam,1


### Exercise 5

- Get all the `Python` or `Scala` related courses in `draft` status.

In [43]:
sql_df = pd.read_sql_query(
    '''
        SELECT *
        FROM
            courses
        WHERE
            course_status = 'draft'
        AND
            (course_name ILIKE '%%Python%%' OR course_name ILIKE '%%Scala%%');
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,course_id,course_name,course_author,course_status,course_published_dt
0,3,Data Engineering using Scala,Elvis Presley,draft,None
1,8,Pipeline Orchestration - Python,Bob Dillon,draft,None


### Exercise 6

- Get the author and count where the author have more than **one published** course.

In [52]:
sql_df = pd.read_sql_query(
    '''
        SELECT
            course_author,
            COUNT(course_status) AS published_courses
        FROM
            courses
        WHERE
            course_status = 'published'
        GROUP BY
            course_author,
            course_status
        HAVING
            COUNT(course_status) > 1
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,course_author,published_courses
0,Bob Dillon,4
